In [1]:
import numpy as np
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from apyori import apriori
from mlxtend.preprocessing import TransactionEncoder
from tqdm import tqdm

In [2]:
queryString = "SELECT * WHERE { ?s ?p ?o. }"
sparql = SPARQLWrapper("http://localhost:3030/MergeAll/sparql")
sparql.setQuery(queryString)

try :
   ret = sparql.query()
   # ret is a stream with the results in XML, see <http://www.w3.org/TR/rdf-sparql-XMLres/>
except :
   pass

In [3]:
sparql.setQuery("""
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT ?s (COUNT(?prop) AS ?total) {
  
  SELECT DISTINCT ?s ?prop
    WHERE {
    ?s wdt:P31 wd:Q515 .
    ?s ?prop ?value .
} 

} GROUP BY ?s
ORDER BY DESC(?total)
limit 1500
""")
sparql.setReturnFormat(JSON)
results_city = sparql.query().convert()

In [4]:
sparql.setQuery("""
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT ?s (COUNT(?prop) AS ?total) {
  
  SELECT DISTINCT ?s ?prop
    WHERE {
    ?s wdt:P31 wd:Q16521 .
    ?s ?prop ?value .
} 

} GROUP BY ?s
ORDER BY DESC(?total)
limit 1500
""")
sparql.setReturnFormat(JSON)
results_taxon = sparql.query().convert()

In [5]:
sparql.setQuery("""
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT ?s (COUNT(?prop) AS ?total) {
  
  SELECT DISTINCT ?s ?prop
    WHERE {
    ?s wdt:P31 wd:Q431289 .
    ?s ?prop ?value .
} 

} GROUP BY ?s
ORDER BY DESC(?total)
limit 1500
""")
sparql.setReturnFormat(JSON)
results_brand = sparql.query().convert()

In [6]:
sparql.setQuery("""
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT ?s (COUNT(?prop) AS ?total) {
  
  SELECT DISTINCT ?s ?prop
    WHERE {
    ?s wdt:P31 wd:Q5 .
    ?s ?prop ?value .
} 

} GROUP BY ?s
ORDER BY DESC(?total)
limit 1500
""")
sparql.setReturnFormat(JSON)
results_human = sparql.query().convert()

In [7]:
sparql.setQuery("""
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT ?s (COUNT(?prop) AS ?total) {
  
  SELECT DISTINCT ?s ?prop
    WHERE {
    ?s wdt:P31 wd:Q2095 .
    ?s ?prop ?value .
} 

} GROUP BY ?s
ORDER BY DESC(?total)
limit 1500
""")
sparql.setReturnFormat(JSON)
results_food = sparql.query().convert()

In [8]:
res_city = []
for results in results_city["results"]["bindings"]:
#     print('%s: %s' % (results["s"]["value"], results["total"]["value"]))
    entity = str(results["s"]["value"]).split('/')
    res_city.append(entity[-1])

In [9]:
db = []

for i in range(len(res_city)):
    query_string = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    SELECT DISTINCT ?s ?prop {
    VALUES ?s {wd:""" + res_city[i] + """}
    ?s ?prop ?value .
    }
    """

    sparql.setQuery(query_string)
    sparql.setReturnFormat(JSON)
    results_entity = sparql.query().convert()
    propLabel = []
    for results in results_entity["results"]["bindings"]:
#         print('%s: %s' % (results["country"]["value"], results["propLabel"]["value"]))
        propLabel.append(results["prop"]["value"])
#     print('---------------------------')
    db.append(propLabel)

In [10]:
te = TransactionEncoder()
te_ary = te.fit(db).transform(db)
df_city = pd.DataFrame(te_ary, columns=te.columns_)
label_city = ["City"] * len(df_city)
df_city["Label"] = label_city
df_city = df_city.rename('{}_City'.format)
df_city

,http://www.wikidata.org/prop/P10,http://www.wikidata.org/prop/P1019,http://www.wikidata.org/prop/P1028,http://www.wikidata.org/prop/P103,http://www.wikidata.org/prop/P1081,http://www.wikidata.org/prop/P1082,http://www.wikidata.org/prop/P112,http://www.wikidata.org/prop/P1151,http://www.wikidata.org/prop/P1198,http://www.wikidata.org/prop/P122,...,http://www.wikidata.org/prop/direct/P85,http://www.wikidata.org/prop/direct/P868,http://www.wikidata.org/prop/direct/P8744,http://www.wikidata.org/prop/direct/P8933,http://www.wikidata.org/prop/direct/P8989,http://www.wikidata.org/prop/direct/P910,http://www.wikidata.org/prop/direct/P912,http://www.wikidata.org/prop/direct/P92,http://www.wikidata.org/prop/direct/P9241,Label
0_City,False,False,False,False,True,True,False,True,True,True,...,True,False,True,False,False,True,False,False,True,City
1_City,False,False,False,False,True,True,False,True,True,False,...,True,False,True,True,True,True,False,True,True,City
2_City,False,False,False,False,False,True,False,True,False,False,...,False,False,True,False,True,True,False,False,True,City
3_City,False,False,False,False,False,True,True,True,False,False,...,False,False,True,False,False,True,False,False,True,City
4_City,False,False,False,False,False,True,False,True,False,False,...,False,False,True,False,False,True,False,False,True,City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495_City,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,City
1496_City,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,City
1497_City,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,City
1498_City,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,City


In [11]:
res_brand = []
for results in results_brand["results"]["bindings"]:
#     print('%s: %s' % (results["s"]["value"], results["total"]["value"]))
    entity = str(results["s"]["value"]).split('/')
    res_brand.append(entity[-1])

In [12]:
db = []

for i in range(len(res_brand)):
    query_string = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    SELECT DISTINCT ?s ?prop {
    VALUES ?s {wd:""" + res_brand[i] + """}
    ?s ?prop ?value .
    }
    """

    sparql.setQuery(query_string)
    sparql.setReturnFormat(JSON)
    results_entity = sparql.query().convert()
    propLabel = []
    for results in results_entity["results"]["bindings"]:
#         print('%s: %s' % (results["country"]["value"], results["propLabel"]["value"]))
        propLabel.append(results["prop"]["value"])
#     print('---------------------------')
    db.append(propLabel)

In [13]:
te_ary = te.fit(db).transform(db)
df_brand = pd.DataFrame(te_ary, columns=te.columns_)
label_brand = ["Brand"] * len(df_brand)
df_brand["Label"] = label_brand
df_brand = df_brand.rename('{}_Brand'.format)
df_brand

,http://www.wikidata.org/prop/P10,http://www.wikidata.org/prop/P101,http://www.wikidata.org/prop/P1019,http://www.wikidata.org/prop/P1037,http://www.wikidata.org/prop/P1041,http://www.wikidata.org/prop/P1056,http://www.wikidata.org/prop/P106,http://www.wikidata.org/prop/P1064,http://www.wikidata.org/prop/P1068,http://www.wikidata.org/prop/P1071,...,http://www.wikidata.org/prop/direct/P8571,http://www.wikidata.org/prop/direct/P86,http://www.wikidata.org/prop/direct/P8693,http://www.wikidata.org/prop/direct/P880,http://www.wikidata.org/prop/direct/P910,http://www.wikidata.org/prop/direct/P92,http://www.wikidata.org/prop/direct/P921,http://www.wikidata.org/prop/direct/P941,http://www.wikidata.org/prop/direct/P972,Label
0_Brand,False,False,False,False,False,True,False,False,False,False,...,True,False,False,False,True,False,False,False,False,Brand
1_Brand,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,Brand
2_Brand,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,Brand
3_Brand,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,Brand
4_Brand,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,Brand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495_Brand,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,Brand
1496_Brand,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Brand
1497_Brand,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Brand
1498_Brand,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Brand


In [14]:
res_food = []
for results in results_food["results"]["bindings"]:
    entity = str(results["s"]["value"]).split('/')
    res_food.append(entity[-1])
db = []
for i in range(len(res_food)):
    query_string = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    SELECT DISTINCT ?s ?prop {
    VALUES ?s {wd:""" + res_food[i] + """}
    ?s ?prop ?value .
    }
    """

    sparql.setQuery(query_string)
    sparql.setReturnFormat(JSON)
    results_food = sparql.query().convert()
    propLabel = []
    for results in results_food["results"]["bindings"]:
        propLabel.append(results["prop"]["value"])
    db.append(propLabel)
te_ary = te.fit(db).transform(db)
df_food = pd.DataFrame(te_ary, columns=te.columns_)
label_food = ["Food"] * len(df_food)
df_food["Label"] = label_food
df_food = df_food.rename('{}_Food'.format)
df_food

,http://www.wikidata.org/prop/P10,http://www.wikidata.org/prop/P105,http://www.wikidata.org/prop/P106,http://www.wikidata.org/prop/P1071,http://www.wikidata.org/prop/P123,http://www.wikidata.org/prop/P1269,http://www.wikidata.org/prop/P131,http://www.wikidata.org/prop/P1343,http://www.wikidata.org/prop/P1348,http://www.wikidata.org/prop/P138,...,http://www.wikidata.org/prop/direct/P495,http://www.wikidata.org/prop/direct/P5008,http://www.wikidata.org/prop/direct/P527,http://www.wikidata.org/prop/direct/P566,http://www.wikidata.org/prop/direct/P61,http://www.wikidata.org/prop/direct/P737,http://www.wikidata.org/prop/direct/P7767,http://www.wikidata.org/prop/direct/P910,http://www.wikidata.org/prop/direct/P931,Label
0_Food,False,True,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,True,False,Food
1_Food,False,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,Food
2_Food,False,False,False,False,False,False,True,False,False,True,...,True,False,True,False,False,False,False,False,False,Food
3_Food,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,Food
4_Food,False,False,False,False,False,False,False,False,False,False,...,True,False,True,False,False,False,False,True,False,Food
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495_Food,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Food
1496_Food,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Food
1497_Food,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Food
1498_Food,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Food


In [15]:
res_taxon = []
for results in results_taxon["results"]["bindings"]:
    entity = str(results["s"]["value"]).split('/')
    res_taxon.append(entity[-1])
db = []
for i in range(len(res_taxon)):
    query_string = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    SELECT DISTINCT ?s ?prop {
    VALUES ?s {wd:""" + res_taxon[i] + """}
    ?s ?prop ?value .
    }
    """

    sparql.setQuery(query_string)
    sparql.setReturnFormat(JSON)
    results_taxon = sparql.query().convert()
    propLabel = []
    for results in results_taxon["results"]["bindings"]:
        propLabel.append(results["prop"]["value"])
    db.append(propLabel)
te_ary = te.fit(db).transform(db)
df_taxon = pd.DataFrame(te_ary, columns=te.columns_)
label_taxon = ["Taxon"] * len(df_taxon)
df_taxon["Label"] = label_taxon
df_taxon = df_taxon.rename('{}_Taxon'.format)
df_taxon

,http://www.wikidata.org/prop/P10,http://www.wikidata.org/prop/P1034,http://www.wikidata.org/prop/P105,http://www.wikidata.org/prop/P1056,http://www.wikidata.org/prop/P1071,http://www.wikidata.org/prop/P1151,http://www.wikidata.org/prop/P1268,http://www.wikidata.org/prop/P1282,http://www.wikidata.org/prop/P1299,http://www.wikidata.org/prop/P131,...,http://www.wikidata.org/prop/direct/P788,http://www.wikidata.org/prop/direct/P789,http://www.wikidata.org/prop/direct/P793,http://www.wikidata.org/prop/direct/P8026,http://www.wikidata.org/prop/direct/P8989,http://www.wikidata.org/prop/direct/P910,http://www.wikidata.org/prop/direct/P927,http://www.wikidata.org/prop/direct/P944,http://www.wikidata.org/prop/direct/P9566,Label
0_Taxon,False,False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,True,False,Taxon
1_Taxon,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,Taxon
2_Taxon,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,True,Taxon
3_Taxon,False,False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,True,False,Taxon
4_Taxon,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,Taxon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495_Taxon,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,Taxon
1496_Taxon,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,Taxon
1497_Taxon,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,Taxon
1498_Taxon,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,Taxon


In [16]:
res_human = []
for results in results_human["results"]["bindings"]:
    entity = str(results["s"]["value"]).split('/')
    res_human.append(entity[-1])
db = []
for i in range(len(res_human)):
    query_string = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    SELECT DISTINCT ?s ?prop {
    VALUES ?s {wd:""" + res_human[i] + """}
    ?s ?prop ?value .
    }
    """

    sparql.setQuery(query_string)
    sparql.setReturnFormat(JSON)
    results_human = sparql.query().convert()
    propLabel = []
    for results in results_human["results"]["bindings"]:
        propLabel.append(results["prop"]["value"])
    db.append(propLabel)
te_ary = te.fit(db).transform(db)
df_human = pd.DataFrame(te_ary, columns=te.columns_)
label_human = ["Human"] * len(df_human)
df_human["Label"] = label_human
df_human = df_human.rename('{}_human'.format)
df_human

,http://www.wikidata.org/prop/P10,http://www.wikidata.org/prop/P1000,http://www.wikidata.org/prop/P101,http://www.wikidata.org/prop/P1019,http://www.wikidata.org/prop/P102,http://www.wikidata.org/prop/P1026,http://www.wikidata.org/prop/P103,http://www.wikidata.org/prop/P1035,http://www.wikidata.org/prop/P1037,http://www.wikidata.org/prop/P1038,...,http://www.wikidata.org/prop/direct/P91,http://www.wikidata.org/prop/direct/P910,http://www.wikidata.org/prop/direct/P937,http://www.wikidata.org/prop/direct/P941,http://www.wikidata.org/prop/direct/P9419,http://www.wikidata.org/prop/direct/P945,http://www.wikidata.org/prop/direct/P9493,http://www.wikidata.org/prop/direct/P9600,http://www.wikidata.org/prop/direct/P97,Label
0_human,False,False,False,False,True,False,True,False,False,True,...,False,True,True,False,False,True,True,False,False,Human
1_human,False,False,True,False,True,False,True,False,False,True,...,False,True,True,False,False,False,False,False,False,Human
2_human,False,False,True,False,False,False,True,False,False,True,...,False,True,True,False,False,False,True,False,True,Human
3_human,False,False,True,False,True,False,True,False,False,True,...,False,True,True,False,False,False,False,False,False,Human
4_human,False,False,True,False,False,False,True,False,False,True,...,False,True,False,False,False,False,False,False,False,Human
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495_human,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,Human
1496_human,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Human
1497_human,False,False,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,Human
1498_human,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Human


In [17]:
merge = [df_city, df_brand, df_food, df_taxon, df_human]
df_merge = pd.concat(merge)
df_merge.Label

0_City         City
1_City         City
2_City         City
3_City         City
4_City         City
              ...  
1495_human    Human
1496_human    Human
1497_human    Human
1498_human    Human
1499_human    Human
Name: Label, Length: 7500, dtype: object

In [18]:
df_merge = df_merge[[c for c in df_merge if c not in ['Label']] 
       + ['Label']]
df_merge = df_merge.fillna(value = False)
df_merge

,http://www.wikidata.org/prop/P10,http://www.wikidata.org/prop/P1019,http://www.wikidata.org/prop/P1028,http://www.wikidata.org/prop/P103,http://www.wikidata.org/prop/P1081,http://www.wikidata.org/prop/P1082,http://www.wikidata.org/prop/P112,http://www.wikidata.org/prop/P1151,http://www.wikidata.org/prop/P1198,http://www.wikidata.org/prop/P122,...,http://www.wikidata.org/prop/direct/P8927,http://www.wikidata.org/prop/direct/P8929,http://www.wikidata.org/prop/direct/P91,http://www.wikidata.org/prop/direct/P937,http://www.wikidata.org/prop/direct/P9419,http://www.wikidata.org/prop/direct/P945,http://www.wikidata.org/prop/direct/P9493,http://www.wikidata.org/prop/direct/P9600,http://www.wikidata.org/prop/direct/P97,Label
0_City,False,False,False,False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,City
1_City,False,False,False,False,True,True,False,True,True,False,...,False,False,False,False,False,False,False,False,False,City
2_City,False,False,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,City
3_City,False,False,False,False,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,City
4_City,False,False,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495_human,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Human
1496_human,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Human
1497_human,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,Human
1498_human,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Human


In [19]:
wikidata = SPARQLWrapper("https://query.wikidata.org/sparql")

In [20]:
df_columns = df_merge.loc[: , df_merge.columns != "Label"]
propList = df_columns.columns.tolist()
for i in range(len(propList)):
    propList[i]=propList[i].split('/')[-1]

In [21]:
propLabel = []

for i in tqdm(range(len(propList))):
    query_string = """
    SELECT DISTINCT ?propLabel {
      VALUES ?p {wdt:""" + propList[i] + """}
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". } 
      ?prop wikibase:directClaim ?p .
    }
    """

    wikidata.setQuery(query_string)
    wikidata.setReturnFormat(JSON)
    results_prop = wikidata.query().convert()
    for results in results_prop["results"]["bindings"]:
#         print('%s: %s' % (results["country"]["value"], results["propLabel"]["value"]))
        propLabel.append(results["propLabel"]["value"])
#     print('---------------------------')

100%|██████████████████████████████████████████████████████████████████████████████| 1250/1250 [15:10<00:00,  1.37it/s]


In [22]:
propLabel_cp = propLabel
propLabel_cp.append("Label")
df_merge_cp = df_merge
df_merge_cp.columns = propLabel_cp
df_merge_cp

,video,web feed URL,donated by,native language,Human Development Index,population,founded by,topic's main Wikimedia portal,unemployment rate,basic form of government,...,kunya,religion or world view,sexual orientation,work location,personal library at,allegiance,artist files at,oral history at,noble title,Label
0_City,False,False,False,False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,City
1_City,False,False,False,False,True,True,False,True,True,False,...,False,False,False,False,False,False,False,False,False,City
2_City,False,False,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,City
3_City,False,False,False,False,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,City
4_City,False,False,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495_human,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Human
1496_human,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Human
1497_human,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,Human
1498_human,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Human


In [23]:
df_merge_cp.to_csv(r'D:\UniversityStuff\FourthTerm\AIDS\GA\MergeAll\merge_property.csv', index = False)